## Addressing dirty data with ticdat

Dirty data is an unloved and often overlooked challenge when building analytical models. A typical assumption is that the input data to a model will somehow magically be clean. In reality, there are any number of reasons why dirty data might be passed as input to your engine. The data might be munged together from different systems, the requirements of your data model might be poorly understood, or a user might be simply pushing your model to its limits via what-if analysis. Regardless of the cause, a professional engine will respond gracefully when passed input data that violates basic integrity checks.

`ticdat` allows for a data scientist to define data integrity checks for 4 different categories of problems (in addition to checking for the correct table and field names).
 1. Duplicate rows (i.e. duplicate primary key entries in the same table).
 1. Data type failures. This checks each column for correct data type, legal ranges for numeric data, acceptable flagging strings, nulls present only for columns that allow null, etc.
 1. Foreign key failures, which check that each record of a child table can cross-reference into the appropriate parent table.
 1. Data predicate failures. This checks each row for conditions more complex than the data type failure checks. For example, a maximum column can not be allowed to be smaller than the minimum column.
 
For a data scientist building optimization engines, `ticdat` provides bulk-query routines that are part of the `TicDatFactory` and `PanDatFactory` classes. The engine builder can customize those routines to look the data integrity problems that are specific to the problem at hand. Another programmer (i.e. a software engineer embedded the optimization engine into a live system) can then call these subroutines to pre-validate the input data prior to running `solve`. 

We briefly tour these routines below. Please consult the docstrings for more information regarding their utility.

We will first discuss an optimization engine built with `TicDatFactory`. If you wise to organize your code around `pandas.DataFrame` objects, then the `PanDatFactory` examples further down will be more interesting to you.

## Integrity checking with TicDatFactory 

In [1]:
import ticdat
from diet import input_schema

First, we quickly check that the csv files in `diet_sample_data` represent clean data. The `ticdat` bulk query routines all return "falsey" results on clean data sets. 

In [2]:
dat = input_schema.csv.create_tic_dat("diet_sample_data")
dat

td: {categories: 4, foods: 9, nutrition_quantities: 36}

In [3]:
any (_ for _ in [input_schema.csv.find_duplicates("diet_sample_data"),
                 input_schema.find_data_type_failures(dat), 
                 input_schema.find_foreign_key_failures(dat), 
                 input_schema.find_data_row_failures(dat)])

False

Next, we examine the `diet_dirty_sample_data` data set, which has been deliberately seeded with dirty data.

We first check for duplicate rows. Note that since the dict-of-dict format that `TicDat` uses will remove any row duplications when representing a data set in memory, we must check for duplications on the csv files directly. Similar duplication checking routines are provided for all the `TicDatFactory` readers.

In [4]:
input_schema.csv.find_duplicates("diet_dirty_sample_data")

{'nutrition_quantities': {('milk', 'fat'): 2}}

`ticdat` is telling us that there are two different records in the Nutrition Quantities table defining the amount of fat in milk. This can be easily confirmed by manually inspecting the "nutrition_quantities.csv" file in the "diet_dirty_sample_data" directory. In a real-world data set, manual inspection would be impossible and such a duplication would be easily overlooked.

In [5]:
dat = input_schema.csv.create_tic_dat("diet_dirty_sample_data")
input_schema.find_data_type_failures(dat)

{TableField(table='nutrition_quantities', field='Quantity'): ValuesPks(bad_values=('',), pks=(('chicken', 'fat'), ('macaroni', 'calories')))}

In [6]:
{tuple(k): v.pks for k, v in input_schema.find_data_type_failures(dat).items()}

{('nutrition_quantities', 'Quantity'): (('chicken', 'fat'),
  ('macaroni', 'calories'))}

`ticdat` is telling us that there are two rows which have bad values in the Quantity field of the Nutrition Quantities table. In both cases, the problem is an empty data cell where a number is expected. The rows with this problem are those which specify the quantity for `('macaroni', 'calories')` and `('chicken', 'fat')`. As before, these two errant rows can easily be double checked by manually examining "nutrition_quantities.csv".

In [7]:
input_schema.find_foreign_key_failures(dat, verbosity="Low")

{('nutrition_quantities', 'foods', ('Food', 'Name')): (('pizza',),
  (('pizza', 'calories'),
   ('pizza', 'sodium'),
   ('pizza', 'protein'),
   ('pizza', 'fat')))}

Here, `ticdat` is telling us that there are 4 records in the Nutrition Quantities table that fail to cross reference with the Foods table. In all 4 cases, it is specifically the "pizza" string in the Food field that fails to find a match from the Name field of the Foods table. If you manually examine "foods.csv", you can see this problem arose because of the Foods table was altered to have a "pizza pie" entry instead of a "pizza" entry.

In [8]:
input_schema.find_data_row_failures(dat)

{TablePredicateName(table='categories', predicate_name='Min Max Check'): ('fat',)}

Here, `ticdat` is telling us that the "Min Max Check" (i.e. the check that `row["Max Nutrition"] >= row["Min Nutrition"]`) failed for the "fat" record of the Categories table. This is easily verified by manual inspection of "categories.csv". 

## Integrity checking with PanDatFactory 

`PanDatFactory` is a sibling to `TicDatFactory`, geared for developers who prefer input tables organized into `pandas.DataFrame` objects. The syntax for creating a `PanDatFactory` object is nearly identical to `TicDatFactory`. Refer to [`netflow_pd.py`](https://github.com/ticdat/ticdat/blob/develop/examples/gurobipy/netflow/netflow_pd.py) for an example.

For expediency, I will close create a `PanDatFactory` clone of the `input_schema` object.

In [9]:
input_schema_pd = input_schema.clone(clone_factory=ticdat.PanDatFactory)

Data ingestion is done similarly as before.

In [10]:
dat_pd = input_schema_pd.csv.create_pan_dat("diet_dirty_sample_data")
dat_pd

pd: {categories: 4, foods: 9, nutrition_quantities: 37}

We will now present the same integrity check results. Because a `DataFrame` can actually store duplicate rows, the duplicate row check is done with the data object itself (as opposed to the directory of csv files).

In [11]:
input_schema_pd.find_duplicates(dat_pd)

{'nutrition_quantities':     Food Category  Quantity
 36  milk      fat       1.0}

The other checks are performed the same for `PanDatFactory` as you did for `TicDatFactory`.

In [12]:
input_schema_pd.find_data_type_failures(dat_pd)

{TableField(table='nutrition_quantities', field='Quantity'):         Food  Category  Quantity
 22   chicken       fat       NaN
 30  macaroni  calories       NaN}

In [13]:
input_schema_pd.find_foreign_key_failures(dat_pd, verbosity="Low")

{('nutrition_quantities',
  'foods',
  ('Food', 'Name')):      Food  Category  Quantity
 12  pizza    sodium     820.0
 13  pizza   protein      15.0
 14  pizza  calories     320.0
 25  pizza       fat      12.0}

In [14]:
input_schema_pd.find_data_row_failures(dat_pd)

{TablePredicateName(table='categories', predicate_name='Min Max Check'):   Name  Min Nutrition  Max Nutrition
 2  fat             70           65.0}

For `PanDatFactory`, the results are all dictionaries, with `pandas.DataFrame` objects as values. I'll deep dive into each result to demonstrate this point.

In [15]:
input_schema_pd.find_duplicates(dat_pd)['nutrition_quantities']

,Food,Category,Quantity
36,milk,fat,1.0


In [16]:
fk_fails = input_schema_pd.find_foreign_key_failures(dat_pd, verbosity="Low")
fk_fails['nutrition_quantities', 'foods', ('Food', 'Name')]

,Food,Category,Quantity
12,pizza,sodium,820.0
13,pizza,protein,15.0
14,pizza,calories,320.0
25,pizza,fat,12.0


In [ ]:
input_schema_pd.find_data_row_failures(dat_pd)